#Bouns project for information retrieval course

Hayfa Al-ajmi submission.

# Notebook contents

1. dataset

2. dictionary

3. posting lists

4. compression

5. conclusion

# Dataset :

the dataset is ~15,000 movies-feedback-tweets from twitter.

In [ ]:
!pip install nltk

import pandas as pd
import numpy as np
import nltk
import sys
import re

from IPython.display import clear_output
from nltk.stem import PorterStemmer as ps
from nltk.tokenize import word_tokenize

nltk.download('punkt')

In [ ]:
df = pd.read_csv("tweets.csv",encoding='latin1')

# dictionary

Tokenize 

without : numbers, punctuations , emails and urls

In [ ]:
empty = ["","."]
Terms = []
def Tokenize(text):
    text = re.sub(r"[^a-zA-Z]"," ",text)
    text = re.sub(r"â.*","",text)
    text = text.replace("\n", "")
    text = text.replace("@", "")
    text = text.replace(".", "")
    text = word_tokenize(text)
    text = [t for t in text if t not in empty]
    Terms.extend(text)

for row in df['Tweets'] :
    Tokenize(row)      

In [ ]:
Terms[5:11]

['first', 'date', 'going', 'on', 'near', 'me']

Size of tokenized terms is 3,407,224 Bytes


In [ ]:
print('{:,}'.format(sys.getsizeof(Terms)),"Bytes")

3,407,224 Bytes


Remove duplicated words => term type

for fixed space the dictinary size is **790,700 Bytes**

In [ ]:
TermsTypes = list(dict.fromkeys(Terms))
print('{:,}'.format(len(TermsTypes)*20),"Bytes")

790,700 Bytes


Sort the terms

In [ ]:
TermsTypes = sorted(TermsTypes)

In [ ]:
TermsTypes[800:805]

['Animator', 'Anime', 'Aniston', 'Anita', 'AnithaSampath']

#Casefolding
--fixed size terms storing--

after Casefolding size is now **614,240 Bytes**

decreased by ~22.3%


In [ ]:
TermsCaseFold = [ term.casefold() for term in TermsTypes ]
TermsCaseFold = list(dict.fromkeys(TermsCaseFold))
TermsCaseFold[50:55]

['admin', 'admire', 'admission', 'adorable', 'adore']

In [ ]:
print('{:,}'.format(len(TermsCaseFold)*20),"Bytes")

614,240 Bytes


#Removing stopwords

--fixed size terms storing--

after Removing stopwords size of dictinary is now **598,220 Bytes**

decreased by ~2.6%

In [ ]:
stopWords = pd.read_csv("https://raw.githubusercontent.com/bbalet/stopwords/master/_stopwords.txt")
stopWords = list(stopWords.iloc[2:,:].values)

TermsCaseFold_without_stopwords = [t for t in TermsCaseFold if t not in stopWords]

In [ ]:
print('{:,}'.format(len(TermsCaseFold_without_stopwords)*20),"Bytes")

598,220 Bytes


# Stemming

--fixed size terms storing--

after Removing stopwords size of dictinary is now **493,460 Bytes**

decreased by ~17.5%

In [ ]:
ps = ps()
TermsStemmed = []
for w in TermsCaseFold_without_stopwords:
    TermsStemmed.append(ps.stem(w))

TermsStemmed = list(dict.fromkeys(TermsStemmed))

In [ ]:
print('{:,}'.format(len(TermsStemmed)*20),"Bytes")

493,460 Bytes


# Posting lists

now lets make the posting list

no need to sort the posting list with this algorithm

the documents are implicitly sorted

In [ ]:
def makePostings(Words,casefold=False):
    global PostingLists
    PostingLists = []
    for number,term in enumerate(Words) :
        print(number,"/",len(Words))
        if casefold :
           PostingList=df['Tweets'].str.casefold().str.contains(term)
        PostingList=df['Tweets'].str.contains(term)   
        PostingList=np.where(PostingList)[0]
        PostingLists.append(PostingList)
        clear_output() 


makePostings(TermsTypes)

In [ ]:
print("term is : ",TermsTypes[400],"\nposting list is :",PostingLists[400][:5])

term is :  Adam 
posting list is : [ 894 2407 3192 4008 4012]


Now the dictionary has the terms without casefolding and without removing the stops words

the size of posting lists is : 4,433,464 Bytes

In [ ]:
def sizeOfPosting():
   Size_Of_Postings = sys.getsizeof(PostingLists)

   for Post in PostingLists :
       Size_Of_Postings += sys.getsizeof(Post)

   print('{:,}'.format(Size_Of_Postings),"Bytes")  

sizeOfPosting()   

After Casefolding Size of postings has been decreased by ~22.2%

In [ ]:
makePostings(TermsCaseFold,True)
sizeOfPosting()

3,448,088 Bytes


After Removing stopwords Size of postings has been decreased by ~2.42%

In [ ]:
makePostings(TermsCaseFold_without_stopwords,True)
sizeOfPosting()

3,364,368 Bytes


After Stemming Size of postings has been decreased by ~17.7%

In [ ]:
makePostings(TermsStemmed,True)
sizeOfPosting()

2,766,304 Bytes


#Compressing 

using :

1. one-String dictinary
2. one-String dictinary with blocking

After storing the terms in one-string

and store reference to the terms the size is **382,173 Bytes**

decreased by ~22.5%

In [ ]:
reference_of_terms = []
dictionary_string = ''

for term in TermsStemmed :
    dictionary_string += term
    reference_of_terms.append(len(dictionary_string)-len(term))
    
print('{:,}'.format(sys.getsizeof(reference_of_terms)+sys.getsizeof(dictionary_string)),"Bytes")    

382,173 Bytes


After applying block concept to one-string dictinary

and store reference to the terms the size is **347,322 Bytes**

decreased by ~9.11%

block size = 8

In [ ]:
reference_of_terms = []
dictionary_string = ''
block = 8

for term in TermsStemmed :
    dictionary_string += str(len(dictionary_string)-len(term)) + term

    block -= 1
    if block == 0 :
       reference_of_terms.append(len(dictionary_string)-len(term))
       block = 8

print('{:,}'.format(sys.getsizeof(reference_of_terms)+sys.getsizeof(dictionary_string)),"Bytes")   

347,322 Bytes


# Conclusion

| memory size in bytes of :> | dictionary | cumulative Percentage | non-postitional posting lists |cumulative Percentage |
| --- | --- | --- | --- | --- |
| No numbers | -- | -- | -- |--|
| Case Folding| 22.3%| 22.3% | 22.2% |22.2% |
| Stop words |2.6%| 24.9% | 2.24% |24.44% |
| Stemming |17.5| 42.4% | 17.7% | 42.14% |
| one-String |22.5%| 64.9% |--|42.14% |
| one-String+blocking=8 |9.11%| 74.01% |--|42.14% |